In [9]:
from bs4 import BeautifulSoup
import urllib.request as req
import urllib
import numpy
import pandas as pd
from pyproj import Proj,transform

#한국환경공단_에어코리아_대기오염통계 현황

In [ ]:
busNum = '360'
key = '%2BjzsSyNtwmcqxUsGnflvs3rW2oceFvhHR8AFkM3ao%2Fw50hwHXgGyPVutXw04uAXvrkoWgkoScvvhlH7jgD4%2FRQ%3D%3D'
url1 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getBusRouteList?serviceKey='+key+'&strSrch='+busNum
res = req.urlopen(url1)

soup = BeautifulSoup(res.read().decode('utf-8'), features="xml") #xml은 무조건 utf-8로 인코딩해야 함.

busRouteId = None
for itemList in soup.find_all('itemList') :
    busRouteId = itemList.find('busRouteId').string
    busRouteNm = itemList.find('busRouteNm').string
    if busRouteNm == busNum :
        break

url2 = 'http://ws.bus.go.kr/api/rest/busRouteInfo/getStaionByRoute?ServiceKey='+key+'&busRouteId='+busRouteId
res = req.urlopen(url2)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")

busPos = []
for itemList in soup.find_all('itemList') :
    gpsY = itemList.find('gpsY').string
    gpsX = itemList.find('gpsX').string

    busPos.append((gpsY, gpsX))

print('[ ' + busNum + '번 버스의 운행 위치 ]')
if len(busPos) != 0 :
    print(busNum + '번 버스 ' + str(len(busPos)) + '대 운행중...')
    for lat,lng in busPos :
        print(lat+','+lng)
else :
    print('현재 운행중인 ' + busNum + '번 버스가 없어요...')


In [ ]:
key = '796143536a756e69313134667752417a'
contentType = 'xml'
startIndex = '1'
endIndex = '100'
url = 'http://openapi.seoul.go.kr:8088/'+key+'/'+contentType+'/LampScpgmtb/'+startIndex+'/'+endIndex+'/'
savename = 'output/edu.xml'
req.urlretrieve(url, savename)

xml = open(savename, 'r', encoding='utf-8').read()
soup = BeautifulSoup(xml, 'xml')

pjList = []
for itemList in soup.find_all('row') :
    up_nm = itemList.find('UP_NM').string
    up_nm = '없음' if up_nm is None else up_nm
    pgm_nm = itemList.find('PGM_NM').string
    pgm_nm = '없음' if pgm_nm is None else pgm_nm
    target_nm = itemList.find('TARGET_NM').string
    target_nm = '없음' if target_nm is None else target_nm
    u_price = itemList.find('U_PRICE').string
    u_price = '없음' if u_price is None else u_price
    pjList.append((up_nm, pgm_nm, target_nm, u_price))

print('[ 서울 청소년 수련관 강좌 리스트 ]')
for up_nm,pgm_nm,target_nm,u_price in pjList : 
    print(up_nm+','+pgm_nm+','+target_nm+','+str(u_price))
    


In [5]:
key = '74474a754173696e3533784e687a65'
contentType = 'xml'
url = 'http://openapi.seoul.go.kr:8088/74474a754173696e3533784e687a65/xml/SpotInfo/1/1000/'
res = req.urlopen(url)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")
print(url)
#print(soup.prettify())
print(soup)

http://openapi.seoul.go.kr:8088/74474a754173696e3533784e687a65/xml/SpotInfo/1/1000/
<?xml version="1.0" encoding="utf-8"?>
<SpotInfo><list_total_count>169</list_total_count><RESULT><CODE>INFO-000</CODE><MESSAGE>정상 처리되었습니다</MESSAGE></RESULT><row><spot_num>A-01</spot_num><spot_nm>성산로(금화터널)</spot_nm><grs80tm_x>195489</grs80tm_x><grs80tm_y>452136</grs80tm_y></row><row><spot_num>A-02</spot_num><spot_nm>사직로(사직터널)</spot_nm><grs80tm_x>196756.776106</grs80tm_x><grs80tm_y>452546.638644</grs80tm_y></row><row><spot_num>A-03</spot_num><spot_nm>자하문로(자하문터널)</spot_nm><grs80tm_x>197216.855046</grs80tm_x><grs80tm_y>454350.990432</grs80tm_y></row><row><spot_num>A-04</spot_num><spot_nm>대사관로(삼청터널)</spot_nm><grs80tm_x>198648.893154</grs80tm_x><grs80tm_y>455200.108465</grs80tm_y></row><row><spot_num>A-05</spot_num><spot_nm>율곡로(안국역)</spot_nm><grs80tm_x>198645.671347</grs80tm_x><grs80tm_y>452937.216603</grs80tm_y></row><row><spot_num>A-06</spot_num><spot_nm>창경궁로(서울여자대학교)</spot_nm><grs80tm_x>199825.89671</grs80

In [12]:
spotinfo = []

for itemList in soup.find_all('row') :
    spot_nm = itemList.find('spot_nm').string   
    grs80tm_x = itemList.find('grs80tm_x').string    
    grs80tm_y = itemList.find('grs80tm_y').string    
    spotinfo.append((spot_nm, grs80tm_x, grs80tm_y))

In [17]:
spotinfo = pd.DataFrame(spotinfo)
spotinfo.columns = ['spotname','tm_x','tm_y']
print(spotinfo)

                spotname           tm_x           tm_y
0              성산로(금화터널)         195489         452136
1              사직로(사직터널)  196756.776106  452546.638644
2            자하문로(자하문터널)  197216.855046  454350.990432
3             대사관로(삼청터널)  198648.893154  455200.108465
4               율곡로(안국역)  198645.671347  452937.216603
..                   ...            ...            ...
164               경부고속도로         202107         443264
165                분당수서로         207716         444241
166          강남순환로(관악터널)         191832         437667
167               서부간선도로         189560         446828
168  잠실로(서울놀이마당앞~잠실트리지움)         208208         445659

[169 rows x 3 columns]


In [ ]:
key = 'd7pOtZHjihMLv0obKwPcxzlRNCwfcmnNMjNVwNc08mxUQMg%2B4EYxcVmqBEjZbjgpB2PpHj5ExgzjIjkoz5IHBQ%3D%3D'
contentType = 'xml'
sido = urllib.parse.quote('서울') #한글을 쿼리문자열로 변환 후 검색
url = 'http://apis.data.go.kr/B552584/ArpltnStatsSvc/getCtprvnMesureSidoLIst?sidoName='+sido+'&searchCondition=DAILY&pageNo=1&numOfRows=100&&returnType=xml&serviceKey='+key
res = req.urlopen(url)
soup = BeautifulSoup(res.read().decode('utf-8'), features="xml")
print(url)
print(soup)


In [ ]:
import pandas as pd
import folium

# 대학교 리스트를 데이터프레임 변환
df = pd.read_excel('./data/미세먼지 측정소별 교통량.xlsx')
df = df[['미세먼지 측정소 명','도로명 주소','위도,경도']]
df['위도,경도'] = df['위도,경도'].replace(" ","")
latf = []
for i in df['위도,경도']:
    latf.append(i.split(','))
latlng = latf
lat = []
lng = []
for i in range(len(latlng)):
    lat.append(latlng[i][0])
    lng.append(latlng[i][1])
df['lat'] = lat
df['lng'] = lng
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

# 대학교 위치정보를 CircleMarker로 표시
for name, lat, lng in zip(df['미세먼지 측정소 명'], df.lat,df.lng):
    folium.CircleMarker([lat,lng],
                        radius=5,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)
    
display(seoul_map)

# 지도를 HTML 파일로 저장하기
#seoul_map.save('output/seoul_colleges2.html')

![NumPy image](images/서울시교통측정소.png)

In [60]:
# 서울 지도 만들기
seoul_map = folium.Map(location=[37.55,126.98], tiles='Stamen Terrain', 
                        zoom_start=12)

 folium.CircleMarker([195489,452136],
                        radius=5,         # 원의 반지름
                        color='brown',         # 원의 둘레 색상
                        fill=True,
                        fill_color='coral',    # 원을 채우는 색
                        fill_opacity=0.7, # 투명도    
                        popup=name
    ).add_to(seoul_map)

IndentationError: unexpected indent (<ipython-input-60-e0faa744671b>, line 5)